# Video Games Sales Data

In [ ]:
import pandas as pd
import seaborn as sns
import missingno as msno
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble  import RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error



plt.rcParams.update({'font.size': "12"})
sns.set_style('white')
df = pd.read_csv("/kaggle/input/video-games-sales/video_games_sales.csv", index_col=0)
df.columns = list(map(lambda d: d.capitalize(),df.columns))
df = df.rename(columns={"Na_sales": "NA_Sales","Eu_sales": "EU_Sales","Jp_sales": "JP_Sales","Other_sales": "Other_Sales","Global_sales": "Global_Sales"}).sort_values('Year')
or_df = df.copy(deep=True)

# Exploring The Data and it's structure

In [ ]:
df.info()
#correct types but some null vals

In [ ]:
num_sum = (df.describe())
num_sum

In [ ]:
#categorical vars
df.drop(columns = list(num_sum.columns)).describe()

In [ ]:
duplicated_video_games_size = df.groupby('Name').size()
duplicated_video_games = df.set_index('Name')[duplicated_video_games_size[df.Name] > 1].sort_index().reset_index()
(duplicated_video_games.describe())
print(duplicated_video_games.head(10))
#So We can have a duplicate game because it can be on multiple platforms ( cross platform games ) a thought i had to confirm

In [ ]:
df.Platform.unique()

In [ ]:
df.Genre.unique()

## Data Cleaning

In [ ]:
df.Publisher.sample(10)

In [ ]:
print(df[df.Publisher == 'Unknown'].sample(10))
df.Publisher = df.Publisher.replace('Unknown',np.nan)
#Discovered that Unknown are a video game company but here it means missing values xD

In [ ]:
msno.matrix(df.sort_values('Year'))
print(df.isna().sum(axis=0))
#no clear relation ship between missing values in year and publisher


In [ ]:
msno.dendrogram(df)
msno.heatmap(df)
#seems there is moderate to low correlation between nullity of years and publisher ( smth have to do with the way data was collected or from the source )


In [ ]:
#try to retrieve the year from the game name
null_years = df[df.Year.isna()]
ny_indexs = null_years[null_years.Name.str.contains(r"\d{4}",regex=True)].index
new_years = pd.Series(null_years.Name.str.extract(r"(\d{4})",expand=False))
print(new_years.loc[ny_indexs]) 
#all are valid years or almost valid , not a bad imputation
df.loc[ny_indexs,"Year"] = new_years.loc[ny_indexs]


In [ ]:
null_years = df[df.Year.isna()]
null_pub_df = df[df.Publisher.isna()]

In [ ]:
#dropping null values due to verry low number < 2% + MAR
df.dropna(inplace=True,how='any')
df = df.astype({'Year': 'int64'})
df.isna().sum()

# Univariate Exploration

## Game Exploration

In [ ]:
print(df.Year.unique())
plt.subplots(figsize=(20,10))
plt.title("Number of game per year and platform")
sns.countplot(data=df,x='Year')
#same game if published to a different platform is counted a new game

In [ ]:
#Drop the years > 2016 due to absent data ( outliers )

print(df[df.Year >= 2017].count().mean())
df = df.drop(df[df.Year >= 2017].index,axis=0)
print(df[df.Year >= 2017].count().mean())



The number of game produced kept increasing each year till 2010 when it keeps decreasing (keep in mind that games made at least 100K) ( possible data issues too  , but i'm not gonna search that and i'll try to get any meaningful insights )

In [ ]:
#let's check at the number of games released that are cross platform ( they are released to more than 1 platform)
repeated = df.groupby('Name').cumcount()
df["cross"] = repeated > 0
cross_platform_games_per_year = df.groupby(['Year','Name']).cross.max().reset_index()
cross_platform_games_per_year = cross_platform_games_per_year.groupby('Year').cross.mean().reset_index()
cross_platform_games_per_year.columns = ['Year','Percentage']
fig,ax = plt.subplots(figsize=(20,10))

sns.barplot(data=cross_platform_games_per_year,x="Year",y='Percentage',ax=ax)

## Let's Discover the platforms

In [ ]:
df.Platform.unique()

In [ ]:
plt.subplots(figsize=(20,10))
plt_counts = df.groupby('Platform').size().sort_values(ascending=False)
plt.title('Number Of Games Per Platform')
sns.barplot(x=plt_counts.index,y=plt_counts.values)

PC is far ahead at the 8 place ! and PS2 at the top and with it PS XBOX 360 and some older consoles like PS and PSP

In [ ]:
#let's see the top platform in each year 
num_per_year_plt = df.groupby(['Year','Platform']).size().reset_index(name='num').set_index('Year')
max_num_per_year_plt = num_per_year_plt.reset_index().groupby('Year').num.max()
max_num_per_year_plt = num_per_year_plt[num_per_year_plt.num == max_num_per_year_plt[num_per_year_plt.index]]
fig,ax = plt.subplots(figsize=(20,10))
for index,vals in enumerate(max_num_per_year_plt.values):
    ax.annotate(vals[0],(max_num_per_year_plt.index[index]+0.12,max_num_per_year_plt.iloc[index].num+5),fontsize=13,rotation=45)
sns.lineplot(data=max_num_per_year_plt,y='num',x=max_num_per_year_plt.index)
sns.scatterplot(data=max_num_per_year_plt,y='num',x=max_num_per_year_plt.index)



So PS2 was in the top for 4 consecutive years but lost the lead later on , and wee that all the the older platforms even from the top ones like DS etc also do nto appear from 2011 and above and gave the place to the new play stations

## Let's take a look at the genre of the games published

In [ ]:
df.Genre.unique()

In [ ]:
df.Genre.describe()

In [ ]:
#let's see the number of games per Genre
games_per_genre = df.groupby("Genre").Name.unique().apply(lambda d: len(d)).sort_values(ascending=False)
plt.subplots(figsize=(20,10))
plt.title("Number Of Games Per Genre")

sns.barplot(x=games_per_genre.index,y=games_per_genre.values)

In [ ]:
#let's plot only the top 4 Action Sports Misc and Role-Playing : 
genre_year = df[df.Genre.isin(["Action","Sports",'Misc','Role-Playing'])].groupby(["Year","Genre"]).size().reset_index()
genre_year = genre_year.rename(columns={0 : 'Count'})
plt.subplots(figsize=(20,10))
plt.title("Games Per Genre Evolution")

sns.lineplot(data=genre_year,x='Year',y='Count',hue='Genre',linewidth=2.5)




In [ ]:
#let's see the density of games per genre
totals_genre = df.groupby('Year').size()
genre_year = genre_year.assign(Percentage =lambda d: d.Count / totals_genre[d.Year].values )
plt.subplots(figsize=(20,10))
plt.title("Games Per Genre Evolution ( Percentage ) ")
genre_year_sum = genre_year.groupby('Year').Percentage.sum().reset_index("Year")
genre_year_sum["Genre"] = "Total"

sns.lineplot(data=pd.concat([genre_year,genre_year_sum]).reset_index(),x='Year',y='Percentage',hue='Genre',linewidth=2.5)



#the action games percentage kept increasing  the others were decreasing 
#those 4 categories are occupying 70% of the game genres as of 2016 

## Let's see the global sales

In [ ]:
print(df.Global_Sales.describe())
print(df.Global_Sales.median())

# max 82 ? there are some outliers to the data
sns.histplot(data=df,x='Global_Sales')
#defintly needs some work

In [ ]:
upper_df = df[df.Global_Sales >= 2 ]
lower_df = df[df.Global_Sales < 2 ]
fig,ax = plt.subplots(figsize=(20,10),nrows=1,ncols=2)
ax0 = ax[0].twinx()
ax1 = ax[1].twinx()

sns.histplot(data=upper_df,x="Global_Sales",ax = ax[0])
sns.kdeplot(data=upper_df,x="Global_Sales",ax =ax0)
sns.kdeplot(data=lower_df,x="Global_Sales",ax = ax1)
sns.histplot(data=lower_df,x="Global_Sales",ax = ax[1])


### Lower Sales : sales < 2

In [ ]:
trans_lower_df = lower_df.set_index(['Year',"Genre","Platform","Publisher","Name","cross"])

trans_lower_df = trans_lower_df.stack().reset_index()
trans_lower_df = trans_lower_df.rename(columns={"level_6": "Region",0: "Sales"})


In [ ]:
plt.subplots(figsize=(20,10))
sns.histplot(data=trans_lower_df[trans_lower_df.Region != "Global_Sales"],x='Sales',hue='Region',binwidth=0.025,multiple="dodge")

In [ ]:
plt.subplots(figsize=(20,10))
sns.histplot(data=trans_lower_df[trans_lower_df.Region != "Global_Sales"],x='Sales',hue='Region',binwidth=0.025,multiple="fill")

In [ ]:
plt.subplots(figsize=(20,10))

sns.kdeplot(data=trans_lower_df[trans_lower_df.Region != "Global_Sales"],x='Sales',hue='Region',multiple="fill")
#just another way to see it but it's the same

In [ ]:
#lets compare with box plots 
plt.subplots(figsize=(20,10))
sns.boxplot(data=trans_lower_df[trans_lower_df.Region != "Global_Sales"],x='Region',y='Sales')

speaking about lower tier : ( < 2 millions )<br>
<ul>
<li> So For Na : you expect a great income with some of the highest incomes (rank 1) </li>
<li> Japan : expect low income but some of the games might get high incomes (rank 3 )</li>
<li> Eu: expect a better income than Japan , but you wont get verry high incomes like NA (2)</li>
<li> Others: expect a  verry low income ; you cannot get high incomes even with great games that made a lot of incomes in japan eu or na (rank 4) </li>
 => the market was dominated by EU and NA and JP too when it comes to the lower tier games

In [ ]:
def get_highest_sales(select_df):
    #fig,axs = plt.subplots(figsize=(20,40),nrows=4,ncols=1)
    #for index,region in enumerate(["NA_Sales","EU_Sales","JP_Sales","Other_Sales","Global_Sales"]):
    
    max_df = select_df.groupby(["Year","Region"]).apply(lambda group: group.nlargest(1,columns=["Sales"],keep='first').drop(columns=["Year","Region"]).iloc[0]).reset_index()
    max_df['Type'] = "max"
    min_df = select_df.groupby(["Year","Region"]).apply(lambda group: group.nsmallest(1,columns=["Sales"],keep='first').drop(columns=["Year","Region"]).iloc[0]).reset_index()
    min_df['Type'] = "min"
    mm_df = select_df.groupby(["Year","Region"]).Sales.agg(["mean","median"])
    mm_df =  mm_df.stack().reset_index().rename(columns={0:"Sales","level_2":"Type"})
    return (pd.concat([max_df,min_df]),mm_df)
    
maxmin_lower_df,mm_lower_df = get_highest_sales(trans_lower_df)



In [ ]:
def minmaxIndividualPlot(minmax_df,mm_df):
    fig,axs = plt.subplots(figsize=(20,40),nrows=5,ncols=1)
    minmax_df = minmax_df.set_index("Year")[["Sales","Type","Region"]]
    mm_df = mm_df.set_index("Year")
    select_df = pd.concat([minmax_df,mm_df])
    for index,region in enumerate(["NA_Sales","EU_Sales","JP_Sales","Other_Sales","Global_Sales"]):
        sns.lineplot(data=select_df[(select_df.Region == region) & (select_df.Type != "max")].reset_index(),x='Year',y='Sales',hue="Type",ax=axs[index])
        axs[index].set_title(region)
        
    fig2,axs = plt.subplots(figsize=(20,30),ncols=1,nrows=3)
    
    for index,tp in enumerate(["median","mean","max"]):
        sns.lineplot(data=select_df[select_df.Type == tp].reset_index(),x='Year',y='Sales',hue="Region",ax=axs[index])
        axs[index].set_title(tp)




minmaxIndividualPlot(maxmin_lower_df,mm_lower_df)

from 2010  and above :
<ul>
<li>Other Sales: Is Increasing and can become a good sector to invest in for lower tier games  </li>
<li>NA: Sales are Decreasing overally but still the leader </li>
<li>EU: Sales are Increasing , might overcome NA </li>
<li>JP: Sales are Descreasing</li>
</ul>
=> EU might dominate the market 

In [ ]:
fig,ax = plt.subplots(figsize=(20,10))
corr_df = df[["NA_Sales","EU_Sales","JP_Sales","Other_Sales"]].corr()
sns.heatmap(data=corr_df,ax=ax,annot =True)
corr_df


NA , Other Sales and EU Are moderatly to strong correlated ( maybe due to social media influence and streamer )
JP is lowly correlated with the others 

### Higher Tier: > 2Million

Took the log scale to get a better visualisation and resuls

In [ ]:
trans_higher_df = upper_df.set_index(['Year',"Genre","Platform","Publisher","Name","cross"])

trans_higher_df = trans_higher_df.stack().reset_index()
trans_higher_df = trans_higher_df.rename(columns={"level_6": "Region",0: "Sales"})
trans_higher_df["oldSales"] = trans_higher_df["Sales"]
trans_higher_df["Sales"] = np.log10(trans_higher_df["Sales"])
cnt = trans_higher_df[trans_higher_df["Sales"] == -np.inf].count().values[0]
print(f'-inf data : ${cnt} :about ${cnt / 5} observations in total' )
print(f"Total ${(upper_df.count() / df.count() * 100).values[0] }%")




In [ ]:
plt.subplots(figsize=(20,10))
sns.histplot(data=trans_higher_df,x='Sales',hue='Region',multiple='dodge')

In [ ]:
plt.subplots(figsize=(20,10))
sns.histplot(data=trans_higher_df,x='Sales',hue='Region',multiple='fill',binwidth=0.1)

In [ ]:
fig,axs = plt.subplots(figsize=(20,20),nrows = 2,ncols=1)
colors = {'NA_Sales' : "blue","Other_Sales":'red',"JP_Sales":'green',"Global_Sales": "pink","EU_Sales": "yellow"}
sns.boxplot(data=trans_higher_df,x='Region',y="Sales",palette=colors,ax=axs[0])
sns.violinplot(data=trans_higher_df.drop(trans_higher_df[trans_higher_df.Sales == -np.inf].index),palette=colors,x='Region',y="Sales",ax=axs[1])

plt.axhline(y = 0, color = 'b', label = 'axvline - full height',linewidth=0.5)


As expected  for the high tier :
<ul>
<li>negative sales are all dominated by mostly "Other  Sales" then "japan" and some "EU" and a small amount of "NA"</li>
<li>EU Sales much higher than japan </li>
<li>Positive Sales are dominated by NA </li>
<li>"Other Sales" have better sales than japan but still some games from japan reach higher sales than "Other Sales"</li>
</ul>


In [ ]:
maxmin_higher_df,mm_higher_df = get_highest_sales(trans_higher_df.drop(trans_higher_df[trans_higher_df.Sales == -np.inf].index))
minmaxIndividualPlot(maxmin_higher_df,mm_higher_df)

Same Behaviour , decreasing JP Sales , Increasing Other Sales well Sales , EU Increasing and almost overcoming NA and NA steady or almost decreasing 

# Some Relations Between the Variables

In [ ]:
#let's see the number of publishers per platform
publishers_per_platform = df.groupby('Platform').Publisher.unique().apply(lambda d: len(d)).sort_values(ascending=False)
plt.subplots(figsize=(20,10))
plt.title("Number Of Publishers Per Platform")
sns.barplot(x=publishers_per_platform.index,y=publishers_per_platform.values)

In [ ]:
published_games_cor = pd.DataFrame({'PubNum' : publishers_per_platform,'GameNum': plt_counts})
published_games_cor.corr()
#0.950846 correlation : so the high number of games produced on the ps2 ds etc might be explained by the high number of publishers that are interseted in those platforms


In [ ]:
logdf = df.copy()
logdf['LNA_Sales'] = np.log10(logdf['NA_Sales'])
logdf['LEU_Sales'] = np.log10(logdf['EU_Sales'])
logdf['LOther_Sales'] = np.log10(logdf['Other_Sales'])
logdf['LJP_Sales'] = np.log10(logdf['JP_Sales'])
sns.pairplot(data=df[['NA_Sales','EU_Sales','Other_Sales','JP_Sales']],height=4)

In [ ]:
trans_df = df.set_index(['Year',"Genre","Platform","Publisher","Name","cross"])

trans_df = trans_df.stack().reset_index()
trans_df = trans_df.rename(columns={"level_6": "Region",0: "Sales"})

In [ ]:
sales_per_platform =  df.groupby("Platform").Global_Sales.agg(["mean","median","std"]).sort_values("mean")
sales_per_platform = sales_per_platform.reset_index().set_index(["std","Platform"]).stack()
sales_per_platform = sales_per_platform.reset_index().rename(columns={'level_2': "desc",0:'values'})


In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
sns.barplot(data=sales_per_platform.reset_index(),
            x="Platform",
            y="values",
            order=sales_per_platform[sales_per_platform.desc=='mean'].sort_values('values',ascending=False).Platform,hue='desc',ax=axs)


In [ ]:
sales_per_publisher =  df.groupby("Publisher").Global_Sales.agg(["mean","median","std"]).sort_values("mean")
sales_per_publisher = sales_per_publisher.reset_index().set_index(["std","Publisher"]).stack()
sales_per_publisher = sales_per_publisher.reset_index().rename(columns={'level_2': "desc",0:'values'})

In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
plt.xticks(rotation=45)
sns.barplot(data=sales_per_publisher.reset_index(),
            x="Publisher",
            y="values",
            order=sales_per_publisher[sales_per_publisher.desc=='mean'].sort_values('values',ascending=False).iloc[0:20].Publisher,hue='desc',ax=axs)


In [ ]:
sales_per_genre =  df.groupby("Genre").Global_Sales.agg(["mean","median","std"]).sort_values("mean")
sales_per_genre = sales_per_genre.reset_index().set_index(["std","Genre"]).stack()
sales_per_genre = sales_per_genre.reset_index().rename(columns={'level_2': "desc",0:'values'})

In [ ]:
fig,axs = plt.subplots(figsize=(20,10))
plt.xticks(rotation=45)
sns.barplot(data=sales_per_genre.reset_index(),
            x="Genre",
            y="values",
            order=sales_per_genre[sales_per_genre.desc=='mean'].sort_values('values',ascending=False).iloc[0:20].Genre,hue='desc',ax=axs)


Despite Being First in the number of games , Action is 6 and platform ,shooter games and role-playing are the ones with the highest median / median sales 